In [1]:
#Imports and API Key
#building in offsets

#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline

from basis_expansions.basis_expansions import (
    Polynomial, LinearSpline)

from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier

import xgboost as xgb

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [2]:
#pulling data from Quandl

data = pd.read_csv('datasources2.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [3]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [4]:
#need to write in special logic to factor in for 3YRT being a daily

treas = dataset['3YRT']
dataset = dataset.drop(columns = ['3YRT'])

In [5]:
treas = treas.resample('MS').mean()

In [6]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [7]:
#readd 3YRT back into data

dataset = dataset.join(treas, how='outer')

In [8]:
#converting GDP quarterly data into monthly - need to convert it so it fills in the following 3 months

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [12]:
offset_dict = {'PMI':1, 'UNR':1, 'YUNR':1,'US_NHOME_SALES':1, 'PART_TIME':1, 'CPPR_PRICE':1, 'HOUS_PERMS':1, 'HOUS_STARTS':1, 'CAP_UTIL':1, 'PERS_SAVINGS':3, 'EXPORTS':3, 'IMPORTS':3, 'TRADE_BALANCE':3, 'US_M2':1, 'US_NHOME_SALES':1,'PPI':1,'CPI':1} #positives shift down, negatives shift up. Goal of this dict is to align data to when it gets released

In [13]:
for entry in offset_dict:
    #print(entry)
    #print(offset_dict[entry])
    dataset[str(entry)] = dataset[str(entry)].shift(offset_dict[entry])

In [14]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)
#dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [15]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [16]:
dataset.shape

(1286, 24)

In [17]:
dataset = dataset[552:]

In [18]:
dataset = dataset[:-12]

In [19]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    if str(col)=='Recession':
        continue
    dataclean.clean_zeros(col, dataset)

In [20]:
dataset['3YRT'] = dataset['3YRT'].fillna(dataset['3YRT'].mean())

In [21]:
dataset.head()

,PMI,UNR,YUNR,CONS_SENT,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,...,3 Month Treasury Yield (Bond Equivalent Basis),Spread,CPPR_PRICE_PXY,HOUS_PERMS_PXY,HOUS_STARTS_PXY,CAP_UTIL_PXY,PERS_SAVINGS_PXY,US_M2_PXY,US_NHOME_SALES_PXY,CPI_PXY
1959-01-01,62.7,6.2,12.0,90.8,1081.0,165.088781,1331.221607,1429.531856,69.480123,8.779086,...,2.88,1.14,1,1,1,1,1,1,1,0
1959-02-01,60.5,6.2,12.1,90.8,1022.0,165.088781,1331.221607,1657.000000,69.480123,8.779086,...,2.76,1.20,1,1,0,1,1,0,1,0
1959-03-01,64.4,6.0,11.6,90.8,973.0,165.088781,1331.221607,1667.000000,69.480123,8.779086,...,2.86,1.13,1,1,0,1,1,0,1,0
1959-04-01,66.9,5.9,11.1,90.8,1102.0,165.088781,1331.221607,1620.000000,69.480123,11.300000,...,3.01,1.11,1,1,0,1,0,0,1,0
1959-05-01,67.1,5.6,11.1,95.3,1086.0,165.088781,1331.221607,1590.000000,69.480123,10.600000,...,2.90,1.41,1,1,0,1,0,0,1,0


In [22]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')
momentum_cols.remove('Recession')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [23]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [27]:
dataset.columns

Index(['PMI', 'UNR', 'YUNR', 'CONS_SENT', 'PART_TIME', 'CPPR_PRICE',
       'HOUS_PERMS', 'HOUS_STARTS', 'CAP_UTIL', 'PERS_SAVINGS',
       ...
       '3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift',
       'Spread_12m_shift', 'CPPR_PRICE_PXY_12m_shift',
       'HOUS_PERMS_PXY_12m_shift', 'CPI_1m_shift', 'PPI_1m_shift',
       'CPI_3m_shift', 'PPI_3m_shift', 'CPI_12m_shift', 'PPI_12m_shift'],
      dtype='object', length=107)

In [28]:
#spline time - splines seriously impede the model, Time Horizon of 1 goes from LL of 4.9 to 8.19, AUC degreades by .04
#stickiness remains

#individual splines

CPPR_PRICE_fit = Pipeline([
    ('CPPR_PRICE', ColumnSelector(name='CPPR_PRICE')),
    ('CPPR_PRICE_spline', LinearSpline(knots=[160]))
])

Spread_fit = Pipeline([
    ('Spread', ColumnSelector(name='Spread')),
    ('Spread_spline', LinearSpline(knots=[0,0.25]))
])

EXPORTS_1m_shift_fit = Pipeline([
    ('EXPORTS_1m_shift', ColumnSelector(name='EXPORTS_1m_shift')),
    ('EXPORT1m_spline', LinearSpline(knots=[700,900]))
])

ThreeYT_1m_shift_fit = Pipeline([
    ('3YT_1m_shift', ColumnSelector(name='3YRT_1m_shift')),
    ('3YT_1m_spline', LinearSpline(knots=[-15]))
])

US_M2_1m_shift = Pipeline([
    ('US_M2_1m_shift', ColumnSelector(name='US_M2_1m_shift')),
    ('US_M2_1m_spline', LinearSpline(knots=[17]))
])

HOME_SALES_3m_shift = Pipeline([
    ('HOME_SALES_3m_shift', ColumnSelector(name='US_NHOME_SALES_3m_shift')),
    ('HOME_SALES_3m_spline', LinearSpline(knots=[-500]))
])

PART_TIME_3m_shift = Pipeline([
    ('PART_TIME_3m_shift', ColumnSelector(name='PART_TIME_3m_shift')),
    ('PART_TIME_3m_spline', LinearSpline(knots=[-160]))
])

CAP_UTIL_3m_shift = Pipeline([
    ('CAP_UTIL_3m_shift', ColumnSelector(name='CAP_UTIL_3m_shift')),
    ('CAP_UTIL_3m_spline', LinearSpline(knots=[-0.8]))
])

EXPORTS_3m_shift = Pipeline([
    ('EXPORTS_3m_shift', ColumnSelector(name='EXPORTS_3m_shift')),
    ('EXPORTS_3m_spline', LinearSpline(knots=[1500,1600]))
])

IMPORTS_3m_shift = Pipeline([
    ('IMPORTS_3m_shift', ColumnSelector(name='IMPORTS_3m_shift')),
    ('IMPORTS_3m_spline', LinearSpline(knots=[2000]))
])

TRADE_BALANCE_3m_shift = Pipeline([
    ('TRADE_BALANCE_3m_shift', ColumnSelector(name='TRADE_BALANCE_3m_shift')),
    ('TRADE_BALANCE_3m_spline', LinearSpline(knots=[-2500]))
])

US_M2_3m_shift = Pipeline([
    ('US_M2_3m_shift', ColumnSelector(name='US_M2_3m_shift')),
    ('US_M2_3m_spline', LinearSpline(knots=[60]))
])

HOME_SALES_12m_shift = Pipeline([
    ('HOME_SALES_12m_shift', ColumnSelector(name='US_NHOME_SALES_12m_shift')),
    ('HOME_SALES_12m_spline', LinearSpline(knots=[60]))
])

PART_TIME_12m_shift = Pipeline([
    ('PART_TIME_12m_shift', ColumnSelector(name='PART_TIME_12m_shift')),
    ('PART_TIME_12m_spline', LinearSpline(knots=[-225, -187.5, -180,-140]))
])

CPPR_PRICE_12m_shift = Pipeline([
    ('CPPR_PRICE_12m_shift', ColumnSelector(name='CPPR_PRICE_12m_shift')),
    ('CPPR_PRICE_12m_spline', LinearSpline(knots=[-30]))
])

CAP_UTIL_12m_shift = Pipeline([
    ('CAP_UTIL_12m_shift', ColumnSelector(name='CAP_UTIL_12m_shift')),
    ('CAP_UTIL_12m_spline', LinearSpline(knots=[-2]))
])

Spread_12m_shift = Pipeline([
    ('Spread_12m_shift', ColumnSelector(name='Spread_12m_shift')),
    ('Spread_12m_spline', LinearSpline(knots=[-1]))
])





#union features together

feature_pipeline = FeatureUnion([
    ('intercept', Intercept()),
    ('CPPR_PRICE_fit', CPPR_PRICE_fit),
    ('Spread_fit', Spread_fit),
    ('EXPORTS_1m_shift_fit', EXPORTS_1m_shift_fit),
    ('ThreeYT_1m_shift_fit', ThreeYT_1m_shift_fit),
    ("US_M2_1m_shift", US_M2_1m_shift),
    ("HOME_SALES_3m_shift", HOME_SALES_3m_shift),
    ("PART_TIME_3m_shift", PART_TIME_3m_shift),
    ("CAP_UTIL_3m_shift", CAP_UTIL_3m_shift),
    ("EXPORTS_3m_shift", EXPORTS_3m_shift),
    ("IMPORTS_3m_shift", IMPORTS_3m_shift),
    ("TRADE_BALANCE_3m_shift", TRADE_BALANCE_3m_shift),
    ("HOME_SALES_12m_shift", HOME_SALES_12m_shift),
    ("PART_TIME_12m_shift", PART_TIME_12m_shift),
    ("CPPR_Price_12m_shift", CPPR_PRICE_12m_shift),
    ("CAP_UTIL_12m_shift", CAP_UTIL_12m_shift),
    ("Spread_12m_shift", Spread_12m_shift)
])


feature_pipeline.fit(dataset)
features = feature_pipeline.transform(dataset)

In [30]:
#dropping columns from OG dataset that were splined

splined_cols = ['CPPR_PRICE','Spread','EXPORTS_1m_shift','3YRT_1m_shift','US_M2_1m_shift','PART_TIME_3m_shift',
'CAP_UTIL_3m_shift',
'EXPORTS_3m_shift',
'IMPORTS_3m_shift',
'US_NHOME_SALES_3m_shift',
'TRADE_BALANCE_3m_shift',
'US_M2_3m_shift',
'US_NHOME_SALES_12m_shift',
'PART_TIME_12m_shift',
'CPPR_PRICE_12m_shift',
'CAP_UTIL_12m_shift',
'Spread_12m_shift']

dataset = dataset.drop(columns = splined_cols)

In [31]:
dataset.shape

(722, 90)

In [32]:
dataset[12:]

,PMI,UNR,YUNR,CONS_SENT,PART_TIME,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,EXPORTS,...,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,CPPR_PRICE_PXY_12m_shift,HOUS_PERMS_PXY_12m_shift,CPI_1m_shift,PPI_1m_shift,CPI_3m_shift,PPI_3m_shift,CPI_12m_shift,PPI_12m_shift
1960-01-01,50.6,5.8,11.3,93.8,1000.0,1331.221607,1601.0,69.480123,9.4,1328.0,...,0.70,1.58,0.0,0.0,0.000000,0.000000,0.341297,-0.630915,1.730104,-0.316456
1960-02-01,58.2,5.3,11.1,93.8,1015.0,1092.000000,1460.0,69.480123,10.1,1376.0,...,0.53,1.30,0.0,-1.0,-0.340136,0.317460,-0.340136,0.000000,1.034483,-0.315457
1960-03-01,61.5,5.2,10.9,93.8,1062.0,1088.000000,1503.0,69.480123,11.0,1493.0,...,0.26,0.52,0.0,-1.0,0.341297,0.000000,0.000000,0.317460,1.730104,-0.315457
1960-04-01,52.3,4.8,10.2,93.8,888.0,955.000000,1109.0,69.480123,10.9,2048.0,...,0.16,0.29,0.0,-1.0,0.000000,0.632911,0.000000,0.952381,1.730104,0.315457
1960-05-01,47.8,5.4,11.5,93.3,1041.0,1016.000000,1289.0,69.480123,10.6,2068.0,...,0.04,0.46,0.0,-1.0,0.340136,0.000000,0.682594,0.632911,1.724138,0.000000
1960-06-01,45.3,5.2,10.9,93.3,988.0,1052.000000,1271.0,69.480123,9.4,2055.0,...,-0.19,-0.77,0.0,-1.0,0.000000,-0.314465,0.340136,0.316456,1.724138,-0.314465
1960-07-01,42.6,5.1,10.7,93.3,966.0,958.000000,1247.0,69.480123,8.4,2199.0,...,-0.50,-0.92,0.0,-1.0,0.338983,0.000000,0.680272,-0.314465,1.718213,0.000000
1960-08-01,44.4,5.4,11.0,97.2,1013.0,999.000000,1197.0,69.480123,10.4,2216.0,...,-0.63,-1.11,0.0,-1.0,0.000000,0.000000,0.338983,-0.314465,1.369863,0.000000
1960-09-01,43.7,5.5,10.8,97.2,1018.0,994.000000,1344.0,69.480123,10.4,2215.0,...,-0.88,-1.61,0.0,-1.0,0.000000,-0.315457,0.338983,-0.315457,1.369863,0.000000
1960-10-01,47.6,5.6,11.4,97.2,1027.0,984.000000,1097.0,69.480123,10.4,2195.0,...,-0.64,-1.80,0.0,-1.0,0.000000,0.000000,0.000000,-0.315457,1.023891,-0.315457


In [33]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
#dataset = dataset.iloc[552:]
#dataset = dataset.iloc[:-2]

#y = y.iloc[552:]
y = dataset['Recession']
dataset = dataset.drop(columns = ['Recession'])
X = dataset

In [34]:
### Data Prep Finished Here ###

In [59]:
y_shift = y.shift(0) #needs to be negative to look forward
y_shift = y_shift.fillna(0)

In [60]:
X_train = X.iloc[12:550]
X_test = X.iloc[550:]
y_train = y_shift.iloc[12:550]
y_test = y_shift.iloc[550:]

In [61]:
model = LogisticRegression(penalty = 'l2', C=2000, max_iter = 100, solver = 'sag') #try throwing in a bigger C than 1
#RidgeClassifier().fit(X, y)
model.fit(X_train, y_train) #fitting model

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=2000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
probs = model.predict_proba(X_test)

In [63]:
y_test.shape

(172,)

In [64]:
X_test.shape

(172, 89)

In [65]:
log_loss(y_test, probs)

0.3218822567960351

In [66]:
roc_auc_score(y_test.values, probs[:,1:])

0.8527590090090089

In [67]:
results = pd.DataFrame(probs)
results['actual'] = y_test.values
results.index = y_test.index
pd.set_option('display.float_format', lambda x: '%.3f' % x)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)

               0     1  actual
2004-11-01 0.955 0.045   0.000
2004-12-01 0.904 0.096   0.000
2005-01-01 0.970 0.030   0.000
2005-02-01 0.976 0.024   0.000
2005-03-01 0.902 0.098   0.000
2005-04-01 0.933 0.067   0.000
2005-05-01 0.958 0.042   0.000
2005-06-01 0.860 0.140   0.000
2005-07-01 0.975 0.025   0.000
2005-08-01 0.918 0.082   0.000
2005-09-01 0.938 0.062   0.000
2005-10-01 0.932 0.068   0.000
2005-11-01 0.929 0.071   0.000
2005-12-01 0.979 0.021   0.000
2006-01-01 0.963 0.037   0.000
2006-02-01 0.882 0.118   0.000
2006-03-01 0.940 0.060   0.000
2006-04-01 0.949 0.051   0.000
2006-05-01 0.792 0.208   0.000
2006-06-01 0.906 0.094   0.000
2006-07-01 0.874 0.126   0.000
2006-08-01 0.903 0.097   0.000
2006-09-01 0.786 0.214   0.000
2006-10-01 0.873 0.127   0.000
2006-11-01 0.882 0.118   0.000
2006-12-01 0.661 0.339   0.000
2007-01-01 0.539 0.461   0.000
2007-02-01 0.716 0.284   0.000
2007-03-01 0.839 0.161   0.000
2007-04-01 0.592 0.408   0.000
2007-05-01 0.758 0.242   0.000
2007-06-